In [8]:
import pandas as pd
import numpy as np
import os
from scipy import stats,signal,fftpack
import math
from pywt import wavedec
import traceback
import sys

params = {}
params['data_path'] = 'Bearing1_2' #目标原始数据文件夹
params['save_path'] = 'datafeature/datafeature1_2.csv' #目标输出文件位置

argvs = ['data_path=Bearing1_2','save_path=datafeature/datafeature1_2.csv'] #与上面同步即可
try:

    for i in range(len(argvs)):
        if i < 1:
            continue
        if argvs[i].split('=')[1] == 'None':
            params[argvs[i].split('=')[0]] = None
        else:
            Type = type(params[argvs[i].split('=')[0]])
            params[argvs[i].split('=')[0]] = Type(argvs[i].split('=')[1])

    columns_list=['time_mean','time_std','time_max','time_min','time_rms','time_ptp','time_median','time_iqr','time_pr','time_skew','time_kurtosis','time_var','time_amp',                'time_smr','time_wavefactor','time_peakfactor','time_pulse','time_margin','freq_mean','freq_std','freq_max','freq_min','freq_rms','freq_median',               'freq_iqr','freq_pr','freq_f2','freq_f3','freq_f4','freq_f5','freq_f6','freq_f7','freq_f8','ener_cA5','ener_cD1','ener_cD2','ener_cD3','ener_cD4',        'ener_cD5','ratio_cA5','ratio_cD1','ratio_cD2','ratio_cD3','ratio_cD4','ratio_cD5']
    columns_list1 = [a + '_vibX' for a in columns_list]
    columns_list2 = [a + '_vibY' for a in columns_list]
    columns_list_final = columns_list1 + columns_list2
#特征提取函数，注意函数的输入参数为csv文件的路径
    def feature_get(filepath):
        col = ['1','2','3','4','vib_x','vib_y']
        dfs = pd.DataFrame(pd.read_csv(filepath,names = col))
        df = dfs.loc[:,['vib_x','vib_y']]
        feature_list = []
        for i in df.columns:
            #----------  time-domain feature,18
            #依次为均值，标准差，最大值，最小值，均方根，峰峰值，中位数，四分位差，百分位差，偏度，峰度，方差，整流平均值，方根幅值，波形因子，峰值因子，脉冲值，裕度
            df_line = df[i]
            time_mean = df_line.mean()
            time_std = df_line.std()
            time_max = df_line.max()
            time_min = df_line.min()
            time_rms = np.sqrt(np.square(df_line).mean())
            time_ptp = time_max-time_min 
            time_median = np.median(df_line)
            time_iqr = np.percentile(df_line,75)-np.percentile(df_line,25)
            time_pr = np.percentile(df_line,90)-np.percentile(df_line,10)
            time_skew = stats.skew(df_line)
            time_kurtosis = stats.kurtosis(df_line)
            time_var = np.var(df_line)
            time_amp = np.abs(df_line).mean()
            time_smr = np.square(np.sqrt(np.abs(df_line)).mean())
            #下面四个特征需要注意分母为0或接近0问题，可能会发生报错
            time_wavefactor = time_rms/time_amp
            time_peakfactor = time_max/time_rms
            time_pulse = time_max/time_amp
            time_margin = time_max/time_smr
            #----------  freq-domain feature,15
            #采样频率25600Hz
            df_fftline = fftpack.fft(df[i])
            freq_fftline = fftpack.fftfreq(len(df[i]),1/25600)
            df_fftline = abs(df_fftline[freq_fftline>=0])
            freq_fftline = freq_fftline[freq_fftline>=0]
            #基本特征,依次为均值，标准差，最大值，最小值，均方根，中位数，四分位差，百分位差
            freq_mean = df_fftline.mean()
            freq_std = df_fftline.std()
            freq_max = df_fftline.max()
            freq_min = df_fftline.min()
            freq_rms = np.sqrt(np.square(df_fftline).mean())
            freq_median = np.median(df_fftline)
            freq_iqr = np.percentile(df_fftline,75)-np.percentile(df_fftline,25)
            freq_pr = np.percentile(df_fftline,90)-np.percentile(df_fftline,10)
            #f2 f3 f4反映频谱集中程度
            freq_f2 = np.square((df_fftline-freq_mean)).sum()/(len(df_fftline)-1)
            freq_f3 = pow((df_fftline-freq_mean),3).sum()/(len(df_fftline)*pow(freq_f2,1.5))
            freq_f4 = pow((df_fftline-freq_mean),4).sum()/(len(df_fftline)*pow(freq_f2,2))
            #f5 f6 f7 f8反映主频带位置
            freq_f5 = np.multiply(freq_fftline,df_fftline).sum()/df_fftline.sum()
            freq_f6 = np.sqrt(np.multiply(np.square(freq_fftline),df_fftline).sum())/df_fftline.sum()
            freq_f7 = np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum())/np.multiply(np.square(freq_fftline),df_fftline).sum()
            freq_f8 = np.multiply(np.square(freq_fftline),df_fftline).sum()/np.sqrt(np.multiply(pow(freq_fftline,4),df_fftline).sum()*df_fftline.sum())
            #----------  timefreq-domain feature,12
            #5级小波变换，最后输出6个能量特征和其归一化能量特征
            cA5, cD5, cD4, cD3, cD2, cD1 = wavedec(df[i], 'db10', level=5)
            ener_cA5 = np.square(cA5).sum()
            ener_cD5 = np.square(cD5).sum()
            ener_cD4 = np.square(cD4).sum()
            ener_cD3 = np.square(cD3).sum()
            ener_cD2 = np.square(cD2).sum()
            ener_cD1 = np.square(cD1).sum()
            ener = ener_cA5 + ener_cD1 + ener_cD2 + ener_cD3 + ener_cD4 + ener_cD5
            ratio_cA5 = ener_cA5/ener
            ratio_cD5 = ener_cD5/ener
            ratio_cD4 = ener_cD4/ener
            ratio_cD3 = ener_cD3/ener
            ratio_cD2 = ener_cD2/ener
            ratio_cD1 = ener_cD1/ener
            feature_list.extend([time_mean,time_std,time_max,time_min,time_rms,time_ptp,time_median,time_iqr,time_pr,time_skew,time_kurtosis,time_var,time_amp,
                             time_smr,time_wavefactor,time_peakfactor,time_pulse,time_margin,freq_mean,freq_std,freq_max,freq_min,freq_rms,freq_median,
                             freq_iqr,freq_pr,freq_f2,freq_f3,freq_f4,freq_f5,freq_f6,freq_f7,freq_f8,ener_cA5,ener_cD1,ener_cD2,ener_cD3,ener_cD4,ener_cD5,
                             ratio_cA5,ratio_cD1,ratio_cD2,ratio_cD3,ratio_cD4,ratio_cD5])

        return feature_list

    files_path = params['data_path']
    print(files_path)
    filelists = os.listdir(files_path) #读取文件，得到文件夹中所有csv文件名
    filelists.sort(key=lambda x:int(x[4:-4])) #按照文件名数字由小到大排序
    features = []
#循环读取文件夹中每个csv文件，对每个csv文件进行特征提取
    for info in filelists:
        file_path = os.path.join(files_path,info)
        #print(file_path)
        fea = feature_get(file_path)
        features.append(fea)
    result = pd.DataFrame(features,columns = columns_list_final)
    result.to_csv(params['save_path'], sep=',', header=True, index=False)
    print("Done.")
    #print(result.shape)
except Exception as e:
    traceback.print_exc()

Bearing1_2
Done.


In [2]:
import pandas as pd
path = 'datafeature/datafeature1_2.csv'  #特征提取后的csv文件路径
df = pd.DataFrame(pd.read_csv(path))
delete_features = ['time_mean_vibX','time_std_vibX'] #需要删除的列名自行加到数组里
df = df.drop(delete_features, axis=1) #特征选择之后的数据

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#import xgboost as xgb
from matplotlib import pyplot
import lightgbm as lgb
#from xgboost import plot_importance
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import sys
import json
import traceback

traindata_path = 'DATA/Bearing1_1_fea.csv'
testdata_path = 'DATA/Bearing1_7_fea.csv'

data1 = pd.DataFrame(pd.read_csv(traindata_path))
data7 = pd.DataFrame(pd.read_csv(testdata_path))

X1 = data1.drop(['Label'], axis=1) 
X7 = data7.drop(['Label'], axis=1)

Y1 = data1.loc[:,['Label']] 
Y7 = data7.loc[:,['Label']] 

In [6]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()
X1_scaler =  X_scaler.fit_transform(X1)
X7_scaler =  X_scaler.fit_transform(X7)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X1_scaler, Y1, test_size=0.3, random_state=123)#划分训练和验证集

In [8]:
lgb_train = lgb.Dataset(X_train, y_train['Label'].values)
lgb_eval = lgb.Dataset(X_test, y_test['Label'].values, reference=lgb_train)

# 将参数写成字典下形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression', # 目标函数
    'metric': 'rmse',  # 评估函数
    'num_leaves': 126,   # 叶子节点数
    'learning_rate': 0.05,  # 学习速率
    'feature_fraction': 0.9, # 建树的特征选择比例
    'bagging_fraction': 0.8, # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}

In [10]:
gbm = lgb.train(params,lgb_train,num_boost_round=500,valid_sets=[lgb_train,lgb_eval],early_stopping_rounds=10)

yp = gbm.predict(X7_scaler, num_iteration=gbm.best_iteration)
yt = Y7['Label'].values

[1]	training's rmse: 7747.22	valid_1's rmse: 7547.99
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 7361.5	valid_1's rmse: 7170.19
[3]	training's rmse: 6996.55	valid_1's rmse: 6812.73
[4]	training's rmse: 6648.32	valid_1's rmse: 6472.26
[5]	training's rmse: 6317.9	valid_1's rmse: 6148.97
[6]	training's rmse: 6003.96	valid_1's rmse: 5841.74
[7]	training's rmse: 5705.89	valid_1's rmse: 5550.58
[8]	training's rmse: 5422.76	valid_1's rmse: 5274.24
[9]	training's rmse: 5154.05	valid_1's rmse: 5011.25
[10]	training's rmse: 4898.63	valid_1's rmse: 4763.11
[11]	training's rmse: 4656.06	valid_1's rmse: 4526.62
[12]	training's rmse: 4425.43	valid_1's rmse: 4302.33
[13]	training's rmse: 4206.79	valid_1's rmse: 4089.73
[14]	training's rmse: 3999.08	valid_1's rmse: 3888.51
[15]	training's rmse: 3801.69	valid_1's rmse: 3697.36
[16]	training's rmse: 3614.11	valid_1's rmse: 3515.79
[17]	training's rmse: 3435.92	valid_1's rmse: 3343.67
[18]	training's rmse: 3266.72	va

In [11]:
from sklearn.metrics import mean_squared_error #均方误差
print("整体RMSE is:")
print(np.sqrt(mean_squared_error(yt/60,yp/60))) #除以60是因为标签为秒，换算成分钟

#评价分数
def scores(y_true, y_pred):
    er = (y_true - y_pred)/60
    def apply_each(x):
        if(x <= 0):
            return np.exp(-np.log(0.5) * (x / 5))
        else:
            return np.exp(np.log(0.5) * (x / 20))
    return 'score', np.array([apply_each(i) for i in er]).mean()*100, True
print("分数 is:")
print(scores(yt,yp))

整体RMSE is:
102.0388933441487
分数 is:
('score', 13.35059190898342, True)
